# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
# Import packages
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1. The central limit theorem applies here because our sample observations are independent (names assigned randomly) and the sample size is well above the rule of thumb of 30-40 observations to have confidence that the sampling distribution of the mean will be normally distributed. The test used will be a two-sample z-test.

2. In this case, the null hypothesis is that there is no difference in the rate of callbacks between the resumes assigned the black-sounding names and the white-sounding names. The alternative is that there is a difference.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

#### Frequentist

In [6]:
# Calculate the number of callbacks for each race
count_call_w = sum(data[data.race=='w'].call)
count_call_b = sum(data[data.race=='b'].call)

# Calculate the total rows for each race
count_tot_w = len(data[data.race=='w'].call)
count_tot_b = len(data[data.race=='b'].call)

# Calculate the proportion of callback for each race
prop_w = count_call_w / count_tot_w
prop_b = count_call_b / count_tot_b

# Calculate the pooled proportion
prop_pool = (prop_w * count_tot_w + prop_b * count_tot_b) / (count_tot_w + count_tot_b)

# Calculate the difference in proportions and SE
prop_diff = np.abs(prop_w-prop_b)
std_error = np.sqrt(prop_pool * (1-prop_pool) * ((1./count_tot_w) + (1./count_tot_b)))

# Calculate z-score
z = prop_diff / std_error

#Assume a confidence level of 0.95
CL = 0.95

#Calculate the margin of error
moe = stats.norm.ppf(1-((1-CL)/2)) * std_error

#Calculate the 95% CI from the difference of proportions
ci_lower, ci_upper = stats.norm.interval(CL, loc=prop_diff, scale=std_error)

#Calculate the p-value 
p_value = (1-stats.norm.cdf(z))*2

In [7]:
print('Margin of error: %s' % (moe))
print('95 Percent Confidence interval: %.8f to %.8f' % (ci_lower, ci_upper))
print('P-Value: %s' % (p_value))

Margin of error: 0.015281631502169232
95 Percent Confidence interval: 0.01675122 to 0.04731449
P-Value: 3.983886837577444e-05


In [8]:
# Perform z-test using statsmodels package
from statsmodels.stats.proportion import proportions_ztest

# Get sample proportion of callbacks for black-sounding names
count_b = np.sum(data.race == 'b')
num_success_b = int(np.sum(data[data.race=='b'].call))
num_fail_b = int(count_b - num_success_b)
prop_b = num_success_b/count_b

# Get sample proportion of callbacks for white-sounding names
count_w = np.sum(data.race == 'w')
num_success_w = int(np.sum(data[data.race=='w'].call))
num_fail_w = int(count_w - num_success_w)
prop_w = num_success_w/count_w

stat, p_val = proportions_ztest([num_success_w, num_success_b], [count_w, count_b])
print('Z-Score={0:0.3f}, P-Value={1:0.3%}'.format(stat, p_val))

Z-Score=4.108, P-Value=0.004%


The null hypothesis that there is no difference between the populations is rejected with a p-value of 0.00004.

#### Bootstrap

In [9]:
# Permutation Testing
def permutation_sample(data1, data2):
    """Draws a permuted sample from data1 and data2."""
    
    x = np.concatenate((data1, data2))
    x_p = np.random.permutation(x)
    d_1 = x_p[:len(data1)]
    d_2 = x_p[len(data1):]
    
    return d_1, d_2

def draw_perm_reps(data1, data2, func, size=1):
    """Computes test statistic func size times from random permutations of data1, data2 datasets."""
    
    p_r = np.empty(size)
    for i in range(size):
        d_1, d_2 = permutation_sample(data1, data2)
        p_r[i] = func(d_1, d_2)
    
    return p_r

def diff_of_proportions(data1, data2):
    """Computes a difference of proportions between two datasets for Bernoulli trials."""
    return abs(np.sum(data1) / len(data1) - np.sum(data2)/ len(data2))

w_call = w.call.astype(int).values
b_call = b.call.astype(int).values

# Draw permutation samples
num_trials=100
prop_bs = draw_perm_reps(w_call, b_call, diff_of_proportions, num_trials)

# P-Value
p_val = np.sum(prop_bs >= abs(prop_w - prop_b)) / len(prop_bs)
print('P-Value={0:0.3%}'.format(p_val))

P-Value=0.000%


## Conclusions

The results of this analysis show there is a significant difference between the rate of callbacks for white-sounding versus black-sounding names, significant at the 0.004% level. A 95% confidence interval implies that white-sounding names get 1.7% to 4.7% more callbacks than black-sounding names.

However, we cannot conclude that this is the most important factor in predicting callback success. That is because we have only looked at one variable in isolation and there may be other more predictive variables. To investigate further would require using multivariate methods. For example, logistic regression may be used to deal with the binary dependent variable while allowing for inclusion of multiple covariates.